In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC

In [2]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
for col in df.select_dtypes(include=['number']).columns:
    df[col] = df[col].fillna(df[col].median())

In [6]:
for col in df.select_dtypes(include=['category','object']).columns:
    df[col]= df[col].fillna(df[col].mode()[0])

In [7]:
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

In [8]:
X = df.drop(columns='survived')
y = df['survived']

In [9]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: survived, Length: 891, dtype: int64

In [10]:
categorical_col = X.select_dtypes(include=['category','object']).columns
numerical_col = X.select_dtypes(include='number').columns
print('Categorical Columns :',categorical_col)
print('Numerical Columns :',numerical_col)

Categorical Columns : Index(['sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive'], dtype='object')
Numerical Columns : Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')


In [11]:
encoder = OneHotEncoder(drop='first', sparse_output= False)
X_col = encoder.fit_transform(X[categorical_col])
X_col

array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 1.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 1., 0., 0.]], shape=(891, 16))

In [12]:
encoded_cat_name = encoder.get_feature_names_out(categorical_col)
encoded_cat_name

array(['sex_male', 'embarked_Q', 'embarked_S', 'class_Second',
       'class_Third', 'who_man', 'who_woman', 'deck_B', 'deck_C',
       'deck_D', 'deck_E', 'deck_F', 'deck_G', 'embark_town_Queenstown',
       'embark_town_Southampton', 'alive_yes'], dtype=object)

<!-- Combine Numerical and Encoded Columns with horizontal stack (hstack)

Numeric Column        |        Encoded Categorical Columns

3 | 22 | 1            |        0 | 1 | 0 -->

In [13]:
#combine numeric and encoded categorical columns
X_combine = np.hstack([X[numerical_col].values, X_col])
X_combine

array([[ 3., 22.,  1., ...,  0.,  1.,  0.],
       [ 1., 38.,  1., ...,  0.,  0.,  1.],
       [ 3., 26.,  0., ...,  0.,  1.,  1.],
       ...,
       [ 3., 28.,  1., ...,  0.,  1.,  0.],
       [ 1., 26.,  0., ...,  0.,  0.,  1.],
       [ 3., 32.,  0., ...,  1.,  0.,  0.]], shape=(891, 21))

In [14]:
final_feactures_name = list(numerical_col) + list(encoded_cat_name)
print("\nFinal Feactures after encoded")
print(final_feactures_name)


Final Feactures after encoded
['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_male', 'embarked_Q', 'embarked_S', 'class_Second', 'class_Third', 'who_man', 'who_woman', 'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'embark_town_Queenstown', 'embark_town_Southampton', 'alive_yes']


In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combine)
X_scaled

array([[ 0.82737724, -0.56573646,  0.43279337, ..., -0.30756234,
         0.61583843, -0.78927234],
       [-1.56610693,  0.66386103,  0.43279337, ..., -0.30756234,
        -1.62380254,  1.2669898 ],
       [ 0.82737724, -0.25833709, -0.4745452 , ..., -0.30756234,
         0.61583843,  1.2669898 ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ..., -0.30756234,
         0.61583843, -0.78927234],
       [-1.56610693, -0.25833709, -0.4745452 , ..., -0.30756234,
        -1.62380254,  1.2669898 ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  3.25137334,
        -1.62380254, -0.78927234]], shape=(891, 21))

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X_combine,y,test_size=0.2,random_state=42)

In [17]:
SVM_model = SVC(kernel='rbf',C=1,gamma='scale')
SVM_model.fit(X_train, y_train)

,C,1
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [18]:
y_predict = SVM_model.predict(X_test)
print("Accuracy :", accuracy_score(y_test,y_predict))
print("\nClassification Report :\n", classification_report(y_test,y_predict))

Accuracy : 0.659217877094972

Classification Report :
               precision    recall  f1-score   support

           0       0.64      0.94      0.76       105
           1       0.76      0.26      0.38        74

    accuracy                           0.66       179
   macro avg       0.70      0.60      0.57       179
weighted avg       0.69      0.66      0.61       179



# Hyper perameter Tunning to inprove Accuracy of SVM

In [27]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'sigmoid']
}
# Grid search with cross-validation
grid_search = GridSearchCV(
    SVC(),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)
# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Evaluate with best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'C': 1, 'gamma': 0.01, 'kernel': 'poly'}

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       105
           1       1.00      1.00      1.00        74

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179

